In [97]:
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [60]:
df = pd.read_csv("data/cleared_train.csv")

In [64]:
df_pca = df.groupby(['Sensor']).mean().drop(columns=['Day', "Unnamed: 0"],axis=1).transpose()
df_pca = df_pca.rename(columns={"Sensor":"Time"})
df_pca

Sensor,Sensor_1,Sensor_10,Sensor_100,Sensor_101,Sensor_102,Sensor_103,Sensor_104,Sensor_105,Sensor_106,Sensor_107,...,Sensor_958,Sensor_959,Sensor_96,Sensor_960,Sensor_961,Sensor_962,Sensor_963,Sensor_97,Sensor_98,Sensor_99
Time_1,0.018636,0.023155,0.008328,0.007599,0.016455,0.012060,0.018191,0.006955,0.019456,0.015433,...,0.019255,0.025540,0.009373,0.021288,0.021727,0.020037,0.022980,0.016089,0.019061,0.011948
Time_2,0.018240,0.022252,0.007456,0.006854,0.015287,0.011548,0.017057,0.006408,0.018508,0.014245,...,0.017930,0.020264,0.008557,0.019082,0.020601,0.018478,0.021678,0.015761,0.017788,0.011846
Time_3,0.019257,0.021544,0.006888,0.006592,0.014439,0.010566,0.016425,0.006028,0.018064,0.012330,...,0.019030,0.020862,0.008085,0.018992,0.020369,0.017861,0.020859,0.014052,0.017645,0.011569
Time_4,0.017563,0.019948,0.006861,0.005825,0.011932,0.009591,0.015003,0.004847,0.016235,0.012390,...,0.019746,0.022680,0.007106,0.017330,0.018502,0.016348,0.019469,0.012321,0.015824,0.010630
Time_5,0.017388,0.020161,0.006016,0.005849,0.011901,0.009234,0.014779,0.004622,0.015932,0.012503,...,0.019042,0.021634,0.006949,0.016366,0.018376,0.015333,0.019380,0.011885,0.015573,0.010257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Time_140,0.024236,0.031430,0.012930,0.012275,0.027235,0.019606,0.025756,0.012018,0.027051,0.021518,...,0.023093,0.028715,0.015629,0.029527,0.030181,0.029027,0.031135,0.024695,0.026923,0.019426
Time_141,0.022207,0.029907,0.012477,0.011401,0.022973,0.018112,0.024153,0.011016,0.024936,0.016033,...,0.024106,0.027830,0.014321,0.027622,0.027224,0.026757,0.029247,0.022567,0.025474,0.016601
Time_142,0.021496,0.027632,0.010717,0.010384,0.020835,0.016388,0.021949,0.009717,0.023294,0.015181,...,0.020819,0.028226,0.013164,0.024634,0.024559,0.023869,0.026294,0.020313,0.023516,0.015303
Time_143,0.020680,0.027330,0.010437,0.010237,0.019988,0.015634,0.021743,0.009394,0.023016,0.016027,...,0.020105,0.027234,0.012529,0.024761,0.024239,0.023975,0.025772,0.021085,0.023004,0.015423


In [72]:
X = df.drop(columns=["Unnamed: 0", "Day"])
y = df["Day"]

In [73]:
fets = X.drop("Sensor", axis=1).columns
sensors = X.Sensor
X_train = df.loc[:, fets].values

In [74]:
X = StandardScaler().fit_transform(X_train)
X = pd.DataFrame(X, columns = fets)

In [75]:
X

,Time_1,Time_2,Time_3,Time_4,Time_5,Time_6,Time_7,Time_8,Time_9,Time_10,...,Time_135,Time_136,Time_137,Time_138,Time_139,Time_140,Time_141,Time_142,Time_143,Time_144
0,-0.232196,-0.621891,-0.490520,-0.330289,-0.364737,-0.229263,-0.239519,-0.355352,-0.175172,-0.238840,...,-0.858966,-0.679798,-0.631901,-0.693973,-0.651974,-0.584317,-0.499363,-0.583953,-0.573618,-0.425320
1,-0.526408,-0.833342,-0.510494,-0.583125,-0.487636,-0.335308,-0.368685,-0.489337,-0.376868,-0.461395,...,-0.753698,-0.806950,-0.869588,-0.836317,-0.575223,-0.632444,-0.751354,-0.353928,-0.633552,-0.667334
2,-0.637554,-0.602067,-0.457228,-0.432790,-0.419359,-0.469632,-0.318454,-0.313041,-0.293408,-0.266659,...,-0.490526,-0.719259,-0.654324,-0.496530,-0.580020,-0.695010,-0.677239,-0.563951,-0.518678,-0.471663
3,0.349688,0.514662,0.548169,0.400887,0.263412,0.548399,0.463721,0.589592,0.680295,0.929576,...,-0.275603,-0.171189,-0.053380,0.086618,0.005206,0.195347,-0.020086,-0.088899,0.025728,-0.075172
4,0.683128,0.765760,0.914374,0.728891,0.850595,0.689792,0.944507,0.956287,0.986316,0.894802,...,0.101609,0.219037,0.130492,0.173861,0.005206,0.103905,0.068852,0.211135,0.145597,0.228633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257116,-0.258348,-0.304713,-0.317405,-0.316622,-0.303288,-0.314099,-0.332805,-0.348300,-0.328183,-0.336208,...,-0.218582,-0.228188,-0.196889,-0.280719,-0.292204,-0.266676,-0.281959,-0.253917,-0.214011,-0.209052
257117,0.970801,1.182056,1.020906,1.009062,1.389984,1.333130,1.009090,1.287723,1.320157,1.339913,...,0.693745,0.701339,0.569988,0.903943,0.609621,0.421546,0.597539,0.761195,0.690002,0.784750
257118,-0.519869,-0.066830,0.055458,-0.309789,-0.118940,-0.349448,0.226916,-0.277782,0.026523,-0.009330,...,-0.433506,-0.508800,-0.282097,-0.551630,-0.196265,-0.415871,-0.173257,-0.178908,-0.428776,-0.147261
257119,0.153547,-0.251850,0.055458,-0.186787,-0.091629,-0.130288,-0.002713,-0.157901,-0.043027,0.004580,...,0.360395,0.447034,0.260547,0.137127,0.312210,0.103905,-0.029968,0.296144,0.255477,0.012365


In [76]:
from sklearn.decomposition import PCA

In [91]:
def createPCAs(components):
    pca = PCA(n_components=components)
    pcas = pca.fit_transform(X_train)
    pca_df = pd.DataFrame(data=pcas, columns=[("pca_" + str(value)) for value in range(1, components+1)])
    return pca_df

In [96]:
pca_2=createPCAs(2)
pca_3=createPCAs(3)
pca_4=createPCAs(4)
pca_5=createPCAs(5)
pca_6=createPCAs(6)
pca_7=createPCAs(7)
pca_8=createPCAs(8)
    

In [102]:
rf = RandomForestClassifier()

RandomForestClassifier()

In [106]:
final_df_2 = pd.concat([pca_2, sensors, y], axis=1)
final_df_3 = pd.concat([pca_3, sensors, y], axis=1)
final_df_4 = pd.concat([pca_4, sensors, y], axis=1)
final_df_5 = pd.concat([pca_5, sensors, y], axis=1)
final_df_6 = pd.concat([pca_6, sensors, y], axis=1)
final_df_7 = pd.concat([pca_7, sensors, y], axis=1)
final_df_8 = pd.concat([pca_8, sensors, y], axis=1)


In [109]:
from sklearn.preprocessing import OrdinalEncoder

In [115]:
ord_enc = OrdinalEncoder()
final_df_8["code_sensor"] = ord_enc.fit_transform(final_df_8[["Sensor"]])
final_df_8.drop("Sensor", axis=1, inplace=True)